In [37]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
199,application_1764662801237_0202,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
164,application_1764662801237_0167,pyspark,idle,Link,Link,None,
170,application_1764662801237_0173,pyspark,idle,Link,Link,None,
174,application_1764662801237_0177,pyspark,idle,Link,Link,None,
180,application_1764662801237_0183,pyspark,idle,Link,Link,None,
183,application_1764662801237_0186,pyspark,idle,Link,Link,None,
187,application_1764662801237_0190,pyspark,idle,Link,Link,None,
188,application_1764662801237_0191,pyspark,idle,Link,Link,None,
192,application_1764662801237_0195,pyspark,idle,Link,Link,None,
198,application_1764662801237_0201,pyspark,idle,Link,Link,None,
199,application_1764662801237_0202,pyspark,idle,Link,Link,None,✔


In [42]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "4g",
        "spark.executor.cores": "2"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
200,application_1764662801237_0203,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
164,application_1764662801237_0167,pyspark,idle,Link,Link,None,
170,application_1764662801237_0173,pyspark,idle,Link,Link,None,
174,application_1764662801237_0177,pyspark,idle,Link,Link,None,
180,application_1764662801237_0183,pyspark,idle,Link,Link,None,
183,application_1764662801237_0186,pyspark,idle,Link,Link,None,
187,application_1764662801237_0190,pyspark,idle,Link,Link,None,
188,application_1764662801237_0191,pyspark,idle,Link,Link,None,
192,application_1764662801237_0195,pyspark,idle,Link,Link,None,
198,application_1764662801237_0201,pyspark,idle,Link,Link,None,
200,application_1764662801237_0203,pyspark,idle,Link,Link,None,✔


In [47]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.memory": "8g",
        "spark.executor.cores": "4"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
201,application_1764662801237_0204,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
164,application_1764662801237_0167,pyspark,idle,Link,Link,None,
170,application_1764662801237_0173,pyspark,idle,Link,Link,None,
174,application_1764662801237_0177,pyspark,idle,Link,Link,None,
180,application_1764662801237_0183,pyspark,idle,Link,Link,None,
183,application_1764662801237_0186,pyspark,idle,Link,Link,None,
187,application_1764662801237_0190,pyspark,idle,Link,Link,None,
188,application_1764662801237_0191,pyspark,idle,Link,Link,None,
192,application_1764662801237_0195,pyspark,idle,Link,Link,None,
198,application_1764662801237_0201,pyspark,idle,Link,Link,None,
201,application_1764662801237_0204,pyspark,idle,Link,Link,None,✔


In [48]:
import time

from sedona.spark import *

from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, count, desc, split, avg, row_number

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
CRIMES_PATH_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv"
CRIMES_PATH_2 = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv"
POLICE_STATION_PATH = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Police_Stations.csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
spark = SparkSession.builder.appName("Query 4").getOrCreate()
sedona = SedonaContext.create(spark)

def load_crime_data(path):
    data = spark.read.csv(path, header=True)
    df = data.select(
        col("DR_NO").cast("integer").alias("id"),
        col("LAT").cast("float").alias("lat1"),
        col("LON").cast("float").alias("lon1")
    ).withColumn("geom1", ST_Point("lon1", "lat1"))
    return df

def load_police_station_data(path):
    data = spark.read.csv(path, header=True)
    df = data.select(
        col("X").cast("float").alias("lon2"),
        col("Y").cast("float").alias("lat2"),
        col("DIVISION").alias("name")
    ).withColumn("geom2", ST_Point("lon2", "lat2"))
    return df

crimes1 = load_crime_data(CRIMES_PATH_1)
crimes2 = load_crime_data(CRIMES_PATH_2)
crimes = crimes1.union(crimes2)

stations = load_police_station_data(POLICE_STATION_PATH)

#stations.show(n=10)
#crimes.show(n=10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
df_start = time.time()

crimes_stations = crimes.join(stations, how="cross") \
    .withColumn("distance", ST_DistanceSphere("geom1", "geom2")/1000)
#crimes_stations.show(10)

# Think of each crime as a partition/group 
# We sort by distance (ascending) so when we use row_number() it gives
# rank 1 to the closest police station
# Meaning that closest_station is a df with crime - closest_station row attributes
window_dist = Window.partitionBy("id").orderBy("distance")
closest_station = crimes_stations.withColumn("rank", row_number().over(window_dist)) \
    .filter(col("rank") == 1)

result = closest_station.groupBy(col("name").alias("division")).agg(
    avg(col("distance")).alias("average_distance"),
    count("*").alias("#")
).orderBy(col("#").desc())

result.show(n=21)

df_end = time.time()
df_time = df_end - df_start
print(f"Execution Time = {df_time}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+------+
|        division|  average_distance|     #|
+----------------+------------------+------+
|       HOLLYWOOD| 2.073555380558326|224124|
|        VAN NUYS| 2.939218980063842|208129|
|       SOUTHWEST| 2.191446044760306|189126|
|        WILSHIRE|2.5933634129262075|186376|
|     77TH STREET| 1.717071770881733|170620|
| NORTH HOLLYWOOD|2.6424535892470318|168096|
|         OLYMPIC|1.7289182568920176|162805|
|         PACIFIC| 3.853507211076137|162027|
|         CENTRAL|0.9933681971560834|154689|
|         RAMPART|1.5342157496329227|153204|
|       SOUTHEAST| 2.443859793883975|143803|
|     WEST VALLEY|3.0215777118913247|136622|
|        FOOTHILL| 4.260093000055929|132482|
|         TOPANGA|3.2970478693696115|131054|
|          HARBOR|3.7016925806773355|127071|
|      HOLLENBECK|333.90901816250255|119381|
|WEST LOS ANGELES| 2.789551816516951|115969|
|          NEWTON| 1.635217212204282|111392|
|       NORTHEAST|3.6229490987035544|108243|
|         

Using 2 executors.

Configuration | Execution Time (s)
--------------|-------------------
1 cores, 2GB mem | 54.816548585891724
2 cores, 4GB mem | 20.3753445148468
4 cores, 8GB mem | 16.06192111968994

In [52]:
result.explain(True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Parsed Logical Plan ==
'Sort ['# DESC NULLS LAST], true
+- Aggregate [name#232], [name#232 AS division#272, avg(distance#250) AS average_distance#284, count(1) AS ##286L]
   +- Filter (rank#261 = 1)
      +- Project [id#98, lat1#99, lon1#100, geom1#105, lon2#230, lat2#231, name#232, geom2#237, distance#250, rank#261]
         +- Project [id#98, lat1#99, lon1#100, geom1#105, lon2#230, lat2#231, name#232, geom2#237, distance#250, rank#261, rank#261]
            +- Window [row_number() windowspecdefinition(id#98, distance#250 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS rank#261], [id#98], [distance#250 ASC NULLS FIRST]
               +- Project [id#98, lat1#99, lon1#100, geom1#105, lon2#230, lat2#231, name#232, geom2#237, distance#250]
                  +- Project [id#98, lat1#99, lon1#100, geom1#105, lon2#230, lat2#231, name#232, geom2#237, ( **org.apache.spark.sql.sedona_sql.expressions.ST_DistanceSphere**   / cast(1000 as double)) AS dis